In [0]:
ecom_df  = spark.read.format("csv")\
    .option("header","true")\
    .option("sep",",")\
    .option("inferSchema","true")\
    .load("/Volumes/resources/ecommerce/ecommerce_etl/data.csv")

ecom_df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [0]:
ecom_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
e

In [0]:
from pyspark.sql.functions import *

ecom_df_clean = ecom_df.dropna(subset=["InvoiceNo", "CustomerID", "Description", "Quantity", "UnitPrice"])

ecom_df_clean = ecom_df_clean.filter((col("Quantity") > 0) & (col("UnitPrice") > 0))

ecom_df_clean = ecom_df_clean.withColumn("Total", round(col("Quantity") * col("UnitPrice"),2))

ecom_df_clean.select("Description","Quantity","Total").show()

+--------------------+--------+-----+
|         Description|Quantity|Total|
+--------------------+--------+-----+
|WHITE HANGING HEA...|       6| 15.3|
| WHITE METAL LANTERN|       6|20.34|
|CREAM CUPID HEART...|       8| 22.0|
|KNITTED UNION FLA...|       6|20.34|
|RED WOOLLY HOTTIE...|       6|20.34|
|SET 7 BABUSHKA NE...|       2| 15.3|
|GLASS STAR FROSTE...|       6| 25.5|
|HAND WARMER UNION...|       6| 11.1|
|HAND WARMER RED P...|       6| 11.1|
|ASSORTED COLOUR B...|      32|54.08|
|POPPY'S PLAYHOUSE...|       6| 12.6|
|POPPY'S PLAYHOUSE...|       6| 12.6|
|FELTCRAFT PRINCES...|       8| 30.0|
|IVORY KNITTED MUG...|       6|  9.9|
|BOX OF 6 ASSORTED...|       6| 25.5|
|BOX OF VINTAGE JI...|       3|14.85|
|BOX OF VINTAGE AL...|       2| 19.9|
|HOME BUILDING BLO...|       3|17.85|
|LOVE BUILDING BLO...|       3|17.85|
|RECIPE BOX WITH M...|       4| 31.8|
+--------------------+--------+-----+
only showing top 20 rows


In [0]:
ecom_df_clean.createOrReplaceTempView("ecommerce_transactions")

Top 10 Selling Products

In [0]:
%sql
SELECT Description,SUM(Quantity) as Total_sold FROM ecommerce_transactions
GROUP BY Description
ORDER BY Total_sold DESC
LIMIT 10;

Description,Total_sold
"PAPER CRAFT , LITTLE BIRDIE",80995
MEDIUM CERAMIC TOP STORAGE JAR,77916
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54415
JUMBO BAG RED RETROSPOT,46181
WHITE HANGING HEART T-LIGHT HOLDER,36725
ASSORTED COLOUR BIRD ORNAMENT,35362
PACK OF 72 RETROSPOT CAKE CASES,33693
POPCORN HOLDER,30931
RABBIT NIGHT LIGHT,27202
MINI PAINT SET VINTAGE,26076


Databricks visualization. Run in Databricks to view.

Top Customers by Revenue

In [0]:
%sql
SELECT CustomerID,ROUND(SUM(Total),2) as Total_spent FROM ecommerce_transactions
GROUP BY CustomerID
ORDER BY Total_spent DESC
LIMIT 10;

CustomerID,Total_spent
14646,280206.02
18102,259657.3
17450,194550.79
16446,168472.5
14911,143825.06
12415,124914.53
14156,117379.63
17511,91062.38
16029,81024.84
12346,77183.6


Monthly Revenue Trend

In [0]:
%sql
SELECT DATE_FORMAT(to_timestamp(InvoiceDate, 'M/d/yyyy H:mm'), 'yyyy-MM') AS Month, 
       ROUND(SUM(Total), 2) AS Revenue
FROM ecommerce_transactions
GROUP BY Month
ORDER BY Month


Month,Revenue
2010-12,572713.89
2011-01,569445.04
2011-02,447137.35
2011-03,595500.76
2011-04,469200.36
2011-05,678594.56
2011-06,661213.69
2011-07,600091.01
2011-08,645343.9
2011-09,952838.38


Revenue by Country

In [0]:
%sql
SELECT Country, ROUND(SUM(Total), 2) AS Revenue
FROM ecommerce_transactions
GROUP BY Country
ORDER BY Revenue DESC
LIMIT 10;

Country,Revenue
United Kingdom,7308391.55
Netherlands,285446.34
EIRE,265545.9
Germany,228867.14
France,209024.05
Australia,138521.31
Spain,61577.11
Switzerland,56443.95
Belgium,41196.34
Sweden,38378.33
